<a href="https://colab.research.google.com/github/vivekverma1019/start-of-line-finder/blob/master/sol_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### mount gdrive and import libraries

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import numpy as np
tf.__version__

'1.15.0'

In [0]:
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [0]:
tf.debugging.set_log_device_placement(True)
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18206171394673884143, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13052276355200035655
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15955913154332834814
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15956161332
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12860394930174473951
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

#### json_state(path)

In [0]:
import time
import json
def json_state(path):
    for i in range(10):
        try:
            with open(path) as f:
                state = json.load(f)
            return state
        except:
            print("Failed to load",i,path)
            time.sleep(i)
            pass
    print("Failed to load state")
    return None

#### generate_random_crop(img , gt , params)

In [0]:

import cv2
import numpy as np
import sys

def perform_crop(img, crop):
    cs = crop['crop_size']
    cropped_gt_img = img[crop['dim0'][0]:crop['dim0'][1], crop['dim1'][0]:crop['dim1'][1]]
    scaled_gt_img = cv2.resize(cropped_gt_img, (cs, cs), interpolation = cv2.INTER_CUBIC)
    return scaled_gt_img


def generate_random_crop(img, gt, params):
    contains_label = np.random.random() < params['prob_label']
    cs = params['crop_size']

    cnt = 0
    while True:

        dim0 = np.random.randint(0,img.shape[0]-cs)
        dim1 = np.random.randint(0,img.shape[1]-cs)
        crop = {
            "dim0": [dim0, dim0+cs],
            "dim1": [dim1, dim1+cs],
            "crop_size": cs
        }
        #TODO: this only works for the center points
        gt_match = np.zeros_like(gt[...,0:2])
        gt_match[...,0][gt[...,0] < dim1] = 1
        gt_match[...,0][gt[...,0] > dim1+cs] = 1
        gt_match[...,1][gt[...,1] < dim0] = 1
        gt_match[...,1][gt[...,1] > dim0+cs] = 1
        gt_match = 1-gt_match
        gt_match = np.logical_and(gt_match[...,0], gt_match[...,1])
        if gt_match.sum() > 0 and contains_label or cnt > 100:
            #print("1st if loop exec.")
            cropped_gt_img = perform_crop(img, crop)
            return crop, cropped_gt_img, np.where(gt_match != 0)

        if gt_match.sum() == 0 and not contains_label:
            #print("second if loop exec.")
            cropped_gt_img = perform_crop(img, crop)
            return crop, cropped_gt_img, np.where(gt_match != 0)
        cnt += 1


In [0]:
import yaml
with open('/content/drive/My Drive/Downloads/start_follow_read/sample_config.yaml') as f:
    config = yaml.load(f)
pretrain_config = config['pretraining']

#### what could be the max no. of sol

In [0]:
tmp = []
for i in range(50):
    if i < 9:
        with open('/content/drive/My Drive/Downloads/start_follow_read/data/train_a/00000'+str(i+1)+'/00000'+str(i+1)+'.json') as gt:
            gt_json = json.load(gt)
            x = [True for i in gt_json if 'sol' in i]
            tmp.append(sum(x))
    else:
        with open('/content/drive/My Drive/Downloads/start_follow_read/data/train_a/0000'+str(i+1)+'/0000'+str(i+1)+'.json') as gt:
            gt_json = json.load(gt)
            x = [True for i in gt_json if 'sol' in i]
            tmp.append(sum(x))
max(tmp)            

63

#### batch_gen()

In [0]:
def batch_gen():
    images = []
    labels = []
    batch_size = 1
    for instance in range(batch_size):
        with open("/content/drive/My Drive/Downloads/start_follow_read/data/train_a_training_set.json") as json_file:
            train_list = json.load(json_file)
        i = np.random.randint(0,len(train_list))
        #print(i)
        org_img = cv2.imread("/content/drive/My Drive/Downloads/start_follow_read/"+ train_list[i][1])
        gt_json = json_state("/content/drive/My Drive/Downloads/start_follow_read/" + train_list[i][0])
        target_dim1 = int(np.random.uniform(384,640))
        s = target_dim1 / float(org_img.shape[1])
        target_dim0 = int(org_img.shape[0]/float(org_img.shape[1]) * target_dim1)
        re_img = cv2.resize(org_img,(target_dim1, target_dim0), interpolation = cv2.INTER_CUBIC)
        gt = np.zeros((1, len(gt_json),4), dtype=np.float32)
        for j , gt_item in enumerate(gt_json):
            #print(gt_item)
            if 'sol' not in gt_item:
                continue
            x0 = gt_item['sol']['x0']
            x1 = gt_item['sol']['x1']
            y0 = gt_item['sol']['y0']
            y1 = gt_item['sol']['y1']

            gt[:,j,0] = x0 * s + 256
            gt[:,j,1] = y0 * s + 256
            gt[:,j,2] = x1 * s + 256
            gt[:,j,3] = y1 * s + 256
        padded_img = np.pad(re_img, ((256,256),(256,256),(0,0)), 'mean')
        pretrain_config = config['pretraining']
        crop_params, patch_img, gt_match = generate_random_crop(padded_img, gt, pretrain_config['sol']['crop_params'])
        patch_img = patch_img.astype(np.float32)
        patch_img = patch_img / 128.0 - 1.0
        gt = gt[gt_match][None,...]
        gt[...,0] = gt[...,0] - crop_params['dim1'][0]
        gt[...,1] = gt[...,1] - crop_params['dim0'][0]
        gt[...,2] = gt[...,2] - crop_params['dim1'][0]
        gt[...,3] = gt[...,3] - crop_params['dim0'][0]
        images.append(patch_img.astype(np.float32))
        gt_padded = np.zeros((1,64,4))
        if gt.shape[1] == 0:
            gt_padded = gt_padded
        else:
            gt_padded[:, 0:gt.shape[1], :] = gt 
        labels.append(gt_padded)
    yield np.array(images) , np.array(labels)

#### dataset object

In [0]:
dataset = tf.data.Dataset.from_generator(batch_gen, output_types=(tf.float32,tf.float32))

Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


#### val_data_gen()

In [0]:
def val_data_gen():
    images = []
    labels = []
    batch_size = 5
    #for instance in range(batch_size):
    with open("/content/drive/My Drive/Downloads/start_follow_read/data/train_a_validation_set.json") as json_file:
        val_list = json.load(json_file)
        for element in val_list:
            org_img = cv2.imread("/content/drive/My Drive/Downloads/start_follow_read/"+ element[1])
            gt_json = json_state("/content/drive/My Drive/Downloads/start_follow_read/" + element[0])
            target_dim1 = int(np.random.uniform(512,512))
            s = target_dim1 / float(org_img.shape[1])
            target_dim0 = int(org_img.shape[0]/float(org_img.shape[1]) * target_dim1)
            re_img = cv2.resize(org_img,(target_dim1, target_dim0), interpolation = cv2.INTER_CUBIC)
            re_img = re_img.astype(np.float32)
            re_img = re_img / 128.0 - 1.0 
            gt = np.zeros((1, len(gt_json),4), dtype=np.float32)
            for j , gt_item in enumerate(gt_json):
                if 'sol' not in gt_item:
                    continue
                x0 = gt_item['sol']['x0']
                x1 = gt_item['sol']['x1']
                y0 = gt_item['sol']['y0']
                y1 = gt_item['sol']['y1']

                gt[:,j,0] = x0 * s
                gt[:,j,1] = y0 * s
                gt[:,j,2] = x1 * s
                gt[:,j,3] = y1 * s
            images.append(re_img)
            labels.append(gt)
        return images , labels
val = val_data_gen()
for cnt , (img, gt) in enumerate(zip(val[0] , val[1])):
    print("{} image shape : {} , gt shape : {}".format(cnt, img.shape , gt.shape))

0 image shape : (809, 512, 3) , gt shape : (1, 45, 4)
1 image shape : (649, 512, 3) , gt shape : (1, 29, 4)
2 image shape : (804, 512, 3) , gt shape : (1, 29, 4)
3 image shape : (805, 512, 3) , gt shape : (1, 21, 4)
4 image shape : (659, 512, 3) , gt shape : (1, 19, 4)


#### model

In [0]:
inputs = tf.keras.Input(shape=(None,None,3))
conv1 = tf.keras.layers.Conv2D(64,kernel_size=3,padding='same',activation='relu' , kernel_initializer = tf.keras.initializers.glorot_normal(), bias_initializer=tf.keras.initializers.Zeros())(inputs)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(conv1)
conv2 = tf.keras.layers.Conv2D(128,kernel_size=3,padding='same',activation='relu', kernel_initializer = tf.keras.initializers.glorot_normal(),bias_initializer=tf.keras.initializers.Zeros())(pool1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(conv2)
conv3 = tf.keras.layers.Conv2D(256,kernel_size=3,padding='same',activation='relu' , kernel_initializer = tf.keras.initializers.glorot_normal(),bias_initializer=tf.keras.initializers.Zeros())(pool2)
conv4 = tf.keras.layers.Conv2D(256,kernel_size=3,padding='same',activation='relu' , kernel_initializer = tf.keras.initializers.glorot_normal(),bias_initializer=tf.keras.initializers.Zeros())(conv3)
pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(conv4)
conv5 = tf.keras.layers.Conv2D(512,kernel_size=3,padding='same',activation='relu' , kernel_initializer = tf.keras.initializers.glorot_normal(),bias_initializer=tf.keras.initializers.Zeros())(pool3)
conv6 = tf.keras.layers.Conv2D(512,kernel_size=3,padding='same',activation='relu' , kernel_initializer = tf.keras.initializers.glorot_normal(),bias_initializer=tf.keras.initializers.Zeros())(conv5)
pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(conv6)
conv7 = tf.keras.layers.Conv2D(512,kernel_size=3,padding='same',activation='relu' , kernel_initializer = tf.keras.initializers.glorot_normal(),bias_initializer=tf.keras.initializers.Zeros())(pool4)
outputs = tf.keras.layers.Conv2D(5,kernel_size=3,padding='same')(conv7)
perm = tf.keras.layers.Permute((3,1,2))(outputs)

Executing op TruncatedNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/rep

In [0]:
def get_predictions(x):
    priors_0 = (tf.range(tf.to_float(tf.shape(x)[2])) + 0.5) * 16
    priors_0 = tf.reshape(priors_0, tf.convert_to_tensor([1, tf.shape(priors_0)[0], 1]))
    priors_0 = tf.broadcast_to(priors_0, shape= tf.convert_to_tensor([1, tf.shape(priors_0)[1], tf.shape(x)[3] ]))
    priors_0 = tf.expand_dims(priors_0, axis=0)
    priors_1 = (tf.range(tf.to_float(tf.shape(x)[3])) + 0.5) * 16
    priors_1 = tf.reshape(priors_1, tf.convert_to_tensor([1, 1, tf.shape(priors_1)[0]]))
    priors_1 = tf.broadcast_to(priors_1, shape= tf.convert_to_tensor([1, tf.shape(x)[2], tf.shape(priors_1)[2]] ))
    priors_1 = tf.expand_dims(priors_1, axis=0)
    predictions = tf.concat( [tf.math.sigmoid(x[:,0:1,:,:]) ,
                             x[:,1:2,:,:] + priors_0 ,
                             x[:,2:3,:,:] + priors_1 ,
                             x[:,3:4,:,:] ,
                             x[:,4:5,:,:] ], axis = 1)
    predictions = tf.transpose(predictions , perm = [0,3,2,1])
    predictions = tf.reshape(predictions, shape = [tf.shape(predictions)[0],-1,5])
    return predictions

In [0]:
second_last_layer_lambda = tf.keras.layers.Lambda(get_predictions)
second_last_layer = second_last_layer_lambda(perm)

Instructions for updating:
Use `tf.cast` instead.


In [0]:
def xyrs_2_xyxy(p):
    x = p[:,:,1:2]
    y = p[:,:,2:3]
    r = p[:,:,3:4]
    s = p[:,:,4:5]
    x0 = -tf.math.sin(r) * s + x
    y0 = -tf.math.cos(r) * s + y
    x1 =  tf.math.sin(r) * s + x
    y1 =  tf.math.cos(r) * s + y
    predictions_xyxy = tf.concat([ p[:,:,0:1],
                              x0, y0, x1, y1 ], axis=2)
    return predictions_xyxy

In [0]:
last_layer_lambda = tf.keras.layers.Lambda(xyrs_2_xyxy)

In [0]:
last_layer = last_layer_lambda(second_last_layer)

In [0]:
model = tf.keras.Model(inputs=inputs, outputs=last_layer)

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 64)    1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, None, None, 64)    0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 128)   73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 128)   0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 256)   295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 256)   590080

#### alignment_loss

In [0]:
def alignment_loss(y_true, y_pred):
    nz_sol_idx = [i for i,j in enumerate(y_true) if np.nonzero(j.numpy())[0].size is not 0]
    z_sol_idx = [i for i,j in enumerate(y_true) if np.nonzero(j.numpy())[0].size is 0]
    if len(z_sol_idx) != 0:
        z_sol_predictions = tf.gather(params = y_pred , indices = z_sol_idx)
        z_sol_confidences = z_sol_predictions[:,:,0]
        z_sol_log_one_minus_confidences = tf.math.log(1.0 -  z_sol_confidences + 1e-10)
        z_sol_loss = tf.math.divide(x = -tf.math.reduce_sum(z_sol_log_one_minus_confidences) , y = len(z_sol_idx))
    if len(nz_sol_idx) != 0:
        nz_sol_predictions = tf.gather(params = y_pred, indices = nz_sol_idx) 
        nz_sol_confidences = nz_sol_predictions[:,:,0]
        nz_sol_log_one_minus_confidences = tf.math.log(1.0 -  nz_sol_confidences + 1e-10)
    else:
        return z_sol_loss
    nz_sol_predictions = tf.expand_dims( nz_sol_predictions, axis=1)
    nz_sol_confidences = tf.expand_dims(nz_sol_confidences , axis=1)
    nz_sol_log_one_minus_confidences = tf.expand_dims(nz_sol_log_one_minus_confidences , axis=1)
    nz_sol_y_true = tf.gather(params = y_true , indices = nz_sol_idx)
    nz_sol_loss = []
    for i in range(len(nz_sol_idx)):
        locations = nz_sol_predictions[i][:,:,1:5]
        y_true = nz_sol_y_true[i]
        int_ten = tf.math.reduce_sum(y_true , axis =2)
        z_v = tf.zeros(shape=(1, y_true.shape[1]) , dtype = tf.float32)
        b_m = tf.not_equal(int_ten , z_v)
        o_z = tf.boolean_mask(y_true , b_m)
        y_true = tf.expand_dims(o_z , axis = 0)
        label_sizes = tf.shape(input = y_true)[1]
        log_confidences = tf.math.log(nz_sol_confidences[i] + 1e-10)
        expanded_locations = tf.expand_dims(locations, axis=2)
        expanded_target = tf.expand_dims(y_true, axis=1)
        expanded_locations = tf.broadcast_to( expanded_locations, shape= [tf.shape(input = locations)[0],
                                                                        tf.shape(input = locations)[1],
                                                                        tf.shape(input = y_true)[1],
                                                                        tf.shape(input = locations)[2]])
        expanded_target = tf.broadcast_to( expanded_target, shape= [tf.shape(input = y_true)[0],
                                                                    tf.shape(input = locations)[1],
                                                                    tf.shape(input = y_true)[1],
                                                                    tf.shape(input = y_true)[2]])
        location_deltas = (expanded_locations - expanded_target)
        normed_difference = tf.norm(location_deltas,ord='euclidean',axis=3)**2
        expanded_log_confidences = tf.expand_dims(log_confidences , axis=2)
        expanded_log_confidences = tf.broadcast_to(expanded_log_confidences , shape=[tf.shape(input = locations)[0],
                                                                                    tf.shape(input = locations)[1],
                                                                                    tf.shape(input = y_true)[1]])
        expanded_log_one_minus_confidences = tf.expand_dims(nz_sol_log_one_minus_confidences[i] , axis=2)
        expanded_log_one_minus_confidences = tf.broadcast_to(expanded_log_one_minus_confidences , shape=[tf.shape(input = locations)[0],
                                                                                                        tf.shape(input = locations)[1],
                                                                                                        tf.shape(input = y_true)[1]])
        alpha_alignment= 0.1
        alpha_backprop= 0.1
        C = alpha_alignment/2.0 * normed_difference - expanded_log_confidences + expanded_log_one_minus_confidences
        X = tf.zeros_like(C)
        X = X.numpy()
        C_i = C[0,:,:label_sizes]
        row_ind, col_ind = linear_sum_assignment(tf.transpose(C_i))
        X[0][(col_ind, row_ind)] = 1.0
        X = tf.Variable(initial_value = X , dtype = tf.float32)
        X2 = 1.0 - tf.math.reduce_sum(X, 2)
        location_loss = tf.math.reduce_sum((alpha_backprop/2.0 * normed_difference * X))
        confidence_loss =  -tf.math.reduce_sum((expanded_log_confidences * X)) - tf.math.reduce_sum((nz_sol_log_one_minus_confidences[i] * X2))
        loss = confidence_loss + location_loss
        nz_sol_loss.append(loss)
    nz_sol_loss = tf.math.divide(x = tf.math.reduce_sum(nz_sol_loss) , y = len(nz_sol_loss))
    if len(z_sol_idx) == 0 :
        return nz_sol_loss
    else :
        return tf.divide(x = z_sol_loss + nz_sol_loss , y = 2)

#### val_loss()


In [0]:
def val_loss():
    validation_loss = []
    for i in range(5):
        #print("image : {} , image shape : {}".format(i , val[0][i].shape))
        #print("gt : {} , gt shape : {}".format(i , val[1][i].shape))
        y_hat = model(tf.expand_dims(tf.transpose(val[0][i] , (1,0,2)) , axis = 0))
        y = tf.expand_dims(val[1][i] , axis = 0) 
        validation_loss.append(alignment_loss(y , y_hat))
    return tf.divide( x = tf.math.reduce_sum(np.array(validation_loss)) , y = 5 )

#### training preparation

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, epsilon = 1e-08)
train_loss_history = []
#validation_loss_history = []
def train_step(image, y_true):
    with tf.GradientTape() as tape:
        y_pred = model(image, training=True)
        loss_value = alignment_loss( y_true, y_pred)
        #print("Train Loss : {}".format(loss_value))
        #print("Validation Loss : {}".format(val_loss()))
    #validation_loss_history.append(val_loss().numpy())
    train_loss_history.append(loss_value.numpy().mean())
    #training_loss.append(loss_value.numpy().mean())
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [0]:
import time
import os
def train(epochs):
    if os.path.exists('/content/drive/My Drive/sol_tf.h5'):
        print("Loading best saved model ...")
        model.load_weights('/content/drive/My Drive/sol_tf.h5')
        print('Validation Loss : {}'.format(val_loss()))
        print("Loaded :-)")
        lowest_loss = val_loss()
    else:
        lowest_loss = np.inf
    cnt_since_last_improvement = 0  
    val_loss_history = []
    for epoch in range(epochs):
        #training_loss = []
        st = time.time()
        for (batch, (image, y_true)) in enumerate(dataset.repeat(1000)):
            print("Epoch : {} , Batch : {}".format(epoch , batch))
            sta = time.time()
            train_step(image, y_true)
            print("Time taken by Epoch {} Batch {} : {} seconds".format(epoch , batch , time.time() - sta ))
            print("ETA at Epoch End : {} seconds ".format( (time.time() - sta) * (1000 - batch + 1) ))
        print ('Epoch {} finished'.format(epoch))
        print("Time taken by Epoch {} : {} seconds".format(epoch , time.time() - st))
        #print("Training Loss : {}".format(sum(training_loss)/1000))
        print("Validation Loss : {}".format(val_loss()))
        val_loss_history.append(val_loss())
        cnt_since_last_improvement += 1
        if lowest_loss > val_loss():
            cnt_since_last_improvement = 0
            lowest_loss = val_loss()
            print("Saving Best")
            model.save('/content/drive/My Drive/sol_tf.h5')
        if cnt_since_last_improvement >= 10:
            break

#### traning

In [0]:
start = time.time()
train(epochs = 1000)
print("Total time taken : {}".format(time.time() - start))

#### tmp.

In [0]:
import seaborn as sns
sns.lineplot(x = list(range(len(train_loss_history))) , y = train_loss_history)

In [0]:
sns.lineplot(x = list(range(len(validation_loss_history))) , y = validation_loss_history)

In [0]:
np.sort(after_predict(model.predict(tf.expand_dims(tf.transpose(val[0][0] , (1,0,2)) , axis = 0)))[...,0:1].numpy(), axis=1)

In [0]:
def after_predict(y_pred):
    batch_size = tf.shape(input = y_pred)[0]
    base_0 , base_1 = tf.constant(16, dtype=tf.float32), tf.constant(16, tf.float32)
    priors_0 = (tf.constant(np.arange(tf.shape(y_pred)[2]) , dtype=tf.float32) + tf.constant(0.5, dtype=tf.float32)) * base_0
    priors_0 = tf.reshape(priors_0, [1, priors_0.shape[0], 1])
    priors_0 = tf.broadcast_to(priors_0, shape=[1, priors_0.shape[1], y_pred.shape[3]])
    priors_0 = tf.expand_dims(priors_0, axis=0)
    #print("priors_0 : {}".format(priors_0))
    priors_1 = (tf.constant(np.arange(tf.shape(y_pred)[3]), dtype = tf.float32) + tf.constant(0.5, dtype=tf.float32)) * base_1
    priors_1 = tf.reshape(priors_1, [1, 1,priors_1.shape[0]])
    priors_1 = tf.broadcast_to(priors_1, shape=[1, y_pred.shape[2], priors_1.shape[2]])
    priors_1 = tf.expand_dims(priors_1, axis=0)
    #print("priors_1 : {}".format(priors_1))
    predictions = tf.concat( [tf.math.sigmoid(y_pred[:,0:1,:,:]) ,
                             y_pred[:,1:2,:,:] + priors_0 ,
                             y_pred[:,2:3,:,:] + priors_1 ,
                             y_pred[:,3:4,:,:] ,
                             y_pred[:,4:5,:,:] ], axis = 1)
    #print("predictions shape: {}".format(predictions.shape))
    predictions = tf.transpose(predictions , perm = [0,3,2,1])
    #print("predictions shape: {}".format(predictions.shape))
    predictions = tf.reshape(predictions, shape = [batch_size,-1,5])
    #print("predictions shape: {}".format(predictions.shape))
    x = predictions[:,:,1:2]
    y = predictions[:,:,2:3]
    r = predictions[:,:,3:4]
    s = predictions[:,:,4:5]
    x0 = -tf.math.sin(r) * s + x
    y0 = -tf.math.cos(r) * s + y
    x1 =  tf.math.sin(r) * s + x
    y1 =  tf.math.cos(r) * s + y
    predictions = tf.concat([ predictions[:,:,0:1],
                              x0, y0, x1, y1 ], axis=2)
    return predictions